In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from multiprocessing import Pool
from collections import namedtuple
from functools import partial


In [6]:
ACC_G = 9.81
def load_data(data_path):
    df = pd.read_csv(data_path)
    data = {
      'roll_lataccel': np.sin(df['roll'].to_numpy()) * ACC_G,
      'v_ego': df['vEgo'].to_numpy(),
      'a_ego': df['aEgo'].to_numpy(),
      'target_lataccel': df['targetLateralAcceleration'].to_numpy(),
      'steer_command': -df['steerCommand'].to_numpy()  # steer commands are logged with left-positive convention but this simulator uses right-positive
    }
    return data

def load_test(file_path, num_segs=5000, processes=1):
    data_path = Path(file_path)
    if data_path.is_file():
        data = load_data(str(data_path))
    elif data_path.is_dir():
        files = sorted(data_path.iterdir())[:num_segs]
        if processes <= 1:
            data = list(map(load_data, [str(file) for file in files]))
        else:
            with Pool(processes) as p:
                data = p.map(load_data, [str(file) for file in files])

    return data

In [42]:
State = namedtuple('State', ['roll_lataccel', 'v_ego', 'a_ego'])
FuturePlan = namedtuple('FuturePlan', ['lataccel', 'roll_lataccel', 'v_ego', 'a_ego'])
FUTURE_PLAN_STEPS = 50

def get_state_target_futureplan(d):

    return [(
        State(roll_lataccel=d['roll_lataccel'][step_idx], v_ego=d['v_ego'][step_idx], a_ego=d['a_ego'][step_idx]),
        d['target_lataccel'][step_idx],
        FuturePlan(
        lataccel=d['target_lataccel'][step_idx + 1:step_idx + FUTURE_PLAN_STEPS],
        roll_lataccel=d['roll_lataccel'][step_idx + 1:step_idx + FUTURE_PLAN_STEPS],
        v_ego=d['v_ego'][step_idx + 1:step_idx + FUTURE_PLAN_STEPS],
        a_ego=d['a_ego'][step_idx + 1:step_idx + FUTURE_PLAN_STEPS]
        )
    ) for step_idx in range(100)]

def get_state_test(data, processes=1):
    if processes <= 1:
        data = list(map(get_state_target_futureplan, data))
    else:
        with Pool(processes) as p:
            data = p.map(get_state_target_futureplan, data)
    return data

In [36]:
file_path_single = './data/00000.csv'
file_path_multi = './data'

In [37]:
data = load_test(file_path_multi, num_segs=5000)

In [2]:
rng = np.random.default_rng()
probs = np.random.rand(512, 1024)
probs /= probs.sum(axis=1, keepdims=True)

In [3]:
%timeit [rng.choice(1024, p = probs[i]) for i in range(512)]

9.03 ms ± 161 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%timeit (probs.cumsum(axis=1) > np.random.rand(probs.shape[0])[:,None]).argmax(axis=1)

1.98 ms ± 105 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
from torch.distributions import Normal
import torch

dist = Normal(torch.tensor([0, 0.]), 1)
sample = dist.sample()
dist.log_prob(sample)

tensor([-1.0235, -1.0810])

In [46]:
a = np.ones((2, 3))
a[0, 2] = -0.1
a[1, 1] = -0.1
a[a < 0] = -1
a

array([[ 1.,  1., -1.],
       [ 1., -1.,  1.]])

In [82]:
a = np.ones((4, 3, 2))
b = np.zeros((4, 2))
a[:, -1]

(4, 2)

In [78]:
d[:, None][:, :, -1] = np.array([0,1,2,3])[:, np.newaxis]
d[:, None]

array([[[0., 0., 0., 1., 0.]],

       [[0., 0., 0., 1., 1.]],

       [[0., 0., 0., 1., 2.]],

       [[0., 0., 0., 1., 3.]]])

In [1]:
[0] * 10

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]